In [3]:
from manim import *
import pandas as pd
import jupyter_capture_output

video_scene = " -v WARNING --disable_caching writing_visualization_Scene"
image_scene = f" -v WARNING --disable_caching -r {2*427},{2*240}  -s writing_visualization_Scene"

In [85]:
# data processing

# total writing data
writing_data = pd.read_csv(f"../external_media/Raw_Sample_Data_Report_Plus_Segs_Report_ERSERO0_segmentiert0.txt", delimiter = "\t", skiprows = 0, skipinitialspace = True).dropna(subset = "seg_path")
# reduce writing data to kinetic energy
kin_energy_writing_data = writing_data.loc[writing_data["seg_path"].str.contains('ERSERO0->kinEnergie')]
kin_energy_writing_data

# pause data


,index,time,x,y,pressure,state,x_filtered,y_filtered,pressure_filtered,x_velocity,...,xy_acceleration,segment_id,status,series_id,run_id,stroke_id,stroke_type,seg_level,seg_path,seg_name
5056,5056,206.940,9246,35384,112,24,9246.0,35384.0,112.0,1.256202,...,-1.0,7,"['FIRST_PRESS', 'PRESSED']",10,68,83,0,2.0,ERSERO0->kinEnergie,E
5057,5057,206.947,9246,35384,592,16,9246.0,35384.0,592.0,0.477689,...,-2.0,7,['PRESSED'],10,68,83,0,2.0,ERSERO0->kinEnergie,E
5058,5058,206.955,9246,35384,1088,16,9246.0,35384.0,1088.0,0.139569,...,0.0,7,['PRESSED'],10,68,83,0,2.0,ERSERO0->kinEnergie,E
5059,5059,206.962,9246,35384,1616,16,9246.0,35384.0,1616.0,0.076590,...,0.0,7,['PRESSED'],10,68,83,0,2.0,ERSERO0->kinEnergie,E
5060,5060,206.970,9246,35384,2128,16,9246.0,35384.0,2128.0,0.181402,...,0.0,7,['PRESSED'],10,68,83,0,2.0,ERSERO0->kinEnergie,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5778,5778,212.371,13756,34857,6707,16,13756.0,34857.0,6707.0,37.380578,...,-1.0,16,['PRESSED'],10,78,91,0,3.0,ERSERO0->kinEnergie->v2,2
5779,5779,212.379,13792,34857,6627,16,13792.0,34857.0,6627.0,33.837746,...,-4.0,16,['PRESSED'],10,78,91,0,3.0,ERSERO0->kinEnergie->v2,2
5780,5780,212.386,13827,34857,6338,16,13827.0,34857.0,6338.0,30.322885,...,-1.0,16,['PRESSED'],10,78,91,0,3.0,ERSERO0->kinEnergie->v2,2
5781,5781,212.394,13856,34857,4610,16,13856.0,34857.0,4610.0,26.814769,...,-6.0,16,['PRESSED'],10,78,91,0,3.0,ERSERO0->kinEnergie->v2,2


In [95]:
class SegPath(Mobject):
    def __init__(self, center, main_height, seg_data, **kwargs):
        super().__init__(**kwargs)
        self.center = center
        self.seg_data = seg_data

        # process writing data
        self.time_data = seg_data["time"]
        self.x_data = seg_data["x"]
        self.y_data = seg_data["y"]
        self.pressure_data = seg_data["pressure"]
        self.seg_name_data = seg_data["seg_name"]

        # set geometry of main frame
        x_data_range = max(self.x_data) - min(self.x_data)
        y_data_range = max(self.y_data) - min(self.y_data)
        print(x_data_range, y_data_range)

        self.main_height = main_height
        self.main_width = float(self.main_height) / y_data_range * x_data_range
        print(self.main_height, self.main_width)

        # main coordinate system
        self.main_npla = NumberPlane(
            x_range = (min(self.x_data), max(self.x_data), 100), y_range = (min(self.y_data), max(self.y_data), 100), x_length = self.main_width, y_length = self.main_height,
            x_axis_config = {"stroke_opacity": 0.215}, y_axis_config = {"stroke_opacity": 0.125}, background_line_style = {"stroke_opacity": 0.125}
        ).move_to(self.center)
        self.add(self.main_npla)


    # returns the dot at position i
    def get_dot(self, time):
        print(time)
        for i in self.seg_data["index"]:
            index_time = int(1000*self.time_data.loc[i])
            pressure = self.pressure_data.loc[i]
            if 1000*time < index_time:
                if pressure < 100:
                    return 0
                else:
                    print(index_time)
                    x_i = self.x_data.loc[i]
                    y_i = self.y_data.loc[i]
                    write_dot = Dot(radius = 0.05, color = RED).move_to(self.main_npla.c2p(x_i, y_i, 0))
                    return write_dot

In [96]:
%%manim -qm --fps 60 $video_scene


class writing_visualization_Scene(Scene):
    def construct(self):

        # main path parameters
        main_path_center = np.array([0, 0, 0])
        main_path_height = 2

        seg_path = SegPath(main_path_center, main_path_height, kin_energy_writing_data)
        self.add(seg_path)

        for time in kin_energy_writing_data["time"]:
            dot = seg_path.get_dot(time)
            if dot:
                self.add(dot)
                self.wait(1.0/60)

Manim Community v0.18.1

4837 1550
2 6.241290322580645
206.94
206947
206.947
206955
206.955
206962
206.962
206970
206.97
206977
206.977
206985
206.985
206992
206.992
207000
207.0
207007
207.007
207015
207.015
207022
207.022
207030
207.03
207037
207.037
207045
207.045
207052
207.052
207060
207.06
207067
207.067
207075
207.075
207082
207.082
207090
207.09
207098
207.098
207105
207.105
207113
207.113
207120
207.12
207128
207.128
207135
207.135
207143
207.143
207150
207.15
207158
207.158
207165
207.165
207173
207.173
207180
207.18
207188
207.188
207195
207.195
207203
207.203
207210
207.21
207218
207.218
207225
207.225
207233
207.233
207240
207.24
207248
207.248
207255
207.255
207263
207.263
207270
207.27
207278
207.278
207285
207.285
207293
207.293
207300
207.3
207308
207.308
207316
207.316
207323
207.323
207.331
207.338
207.346
207.353
207.361
207.368
207.376
207.383
207.391
207.398
207.406
207.413
207.421
207.428
207.436
207.443
207.451
207.458
207.466
207.473
207.481
207.488
207.496
207.503
207.511
207.519
207.